In [ ]:
import sys
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install ipython-autotime
!{sys.executable} -m pip install pandas-profiling

In [ ]:
from datetime import datetime
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re
import joblib

import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action = 'ignore', category = SettingWithCopyWarning)

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import mean_absolute_error
import os

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
plt.style.use('classic')
%matplotlib inline

from pandas_profiling import ProfileReport

from config import *
from utils import *

# Load Data

In [ ]:
path = "./Data"

df_raw = pd.read_csv('{}/nb_purchase_propensity_model_raw_data.csv000'.format(path))
df_policy = pd.read_csv('{}/nb_purchase_propensity_model_raw_data_compliment_view1.csv000'.format(path))
df_quote = pd.read_csv('{}/nb_purchase_propensity_model_raw_data_compliment_view2.csv000'.format(path))

def nric_check(row):
        #     print(type(row['nric']))
        if len(str(row['nric'])) == 9 and str(row['nric'])[:1] in ['S', 'T']:
            return 'SGPR'
        elif len(str(row['nric'])) == 9 and str(row['nric'])[:1] in ['F', 'G']:
            return 'FOREIGNER'
        else:
            return 'UNKNOWN'

In [ ]:
df_raw_quote = df_raw.merge(df_quote, left_on='quote_origin_system_number', right_on='current_quote_origin_system_number', how='left', indicator=True)
df_raw_quote = df_raw_quote.rename(columns={"_merge": "matched_quote"})
df_combined = df_raw_quote.merge(df_policy, left_on='quote_origin_system_number', right_on='current_quote_origin_system_number', how='left', indicator=True)
df_combined = df_combined.rename(columns={"_merge": "matched_policy"})
df_combined = df_combined.drop(columns=['product_type_y', 'current_quote_origin_system_number_x', 'current_quote_origin_system_number_y'])
df_combined = df_combined.rename(columns={"applicant_nric_x": "nric", "applicant_nric_y": "nric_quote", "individual_id_number": "nric_policy", "product_type_x": "product_type",})
df_combined['nric_type'] = df_combined.apply(nric_check, axis=1)


In [ ]:
#dates
df_combined['applicant_dob']= pd.to_datetime(df_combined['applicant_dob'], errors = 'coerce')
df_combined['age_at_quote'] = np.floor((df_combined['quote_issue_date'] - df_combined['applicant_dob'])/np.timedelta64(1, 'Y'))
df_combined['update_timediff'] = np.floor((df_combined['quote_latest_update_date'] - df_combined['quote_issue_date'])/np.timedelta64(1, 'm'))
df_combined['conversion_timediff'] = np.floor((df_combined['policy_issue_date'] - df_combined['quote_issue_date'])/np.timedelta64(1, 'D'))
df_combined['since_first_quote'] = np.floor((df_combined['quote_issue_date'] - df_combined['first_quote_created_before_date'])/np.timedelta64(1, 'W'))
df_combined['since_latest_quote'] = np.floor((df_combined['quote_issue_date'] - df_combined['latest_quote_created_before_date'])/np.timedelta64(1, 'W'))
df_combined['since_first_policy'] = np.floor((df_combined['quote_issue_date'] - df_combined['scv_first_policy_issue_date'])/np.timedelta64(1, 'W'))
df_combined['since_latest_policy'] = np.floor((df_combined['quote_issue_date'] - df_combined['scv_latest_policy_issue_date'])/np.timedelta64(1, 'W'))
df_combined['since_first_interaction'] = np.floor((df_combined['first_interaction_date'] - df_combined['quote_issue_date'])/np.timedelta64(1, 'h'))
df_combined['since_last_interaction'] = np.floor((df_combined['last_interaction_date'] - df_combined['quote_issue_date'])/np.timedelta64(1, 'h'))
df_combined['update_flag'] = np.where(np.floor((df_combined['quote_latest_update_date'] - df_combined['quote_issue_date'])/np.timedelta64(1, 'm')) > 0, 1, 0)
df_combined['multiple_interaction_flag'] = np.where(np.floor((df_combined['last_interaction_date'] - df_combined['first_interaction_date'])/np.timedelta64(1, 's')) > 0, 1, 0)

In [ ]:
#call counts
df_combined['customer_request_call_count'] = df_combined['customer_request_call_count'].fillna(0)
df_combined['customer_request_no_pick_up_count'] = df_combined['customer_request_no_pick_up_count'].fillna(0)
df_combined['customer_request_call_attempt_count'] = (df_combined['customer_request_call_count'] + df_combined['customer_request_no_pick_up_count']).fillna(0)


In [ ]:
# removing unknown NRICs
df_cleaned = df_combined[df_combined['nric_type'] != 'UNKNOWN']

# removing STP
df_cleaned = df_cleaned[df_cleaned['stp_flag'] == 0]

# removing issued before interaction
df_cleaned = df_cleaned[df_cleaned['issued_before_interaction_flag'] == 0]

# removing assisted quotes
df_cleaned = df_cleaned[df_cleaned['assisted_call_flag'] == "Outbound - Assisted"]

# consolidating product types
mapping_product = {'Heart Attack Insurance': 'Big 3 Critical Illness', 'Term Life Plus': 'Life', 'Term Life Plus': 'Life', 'DPI': 'Life', 'Cancer': 'Big 3 Critical Illness',
                   'TermLife': 'Life', 'Heart Attack Insurance-Stroke Insurance': 'Big 3 Critical Illness', 'Stroke Insurance': 'Big 3 Critical Illness', 
                   'Cancer-Heart Attack Insurance': 'Big 3 Critical Illness', 'Cancer-Heart Attack Insurance-Stroke Insurance': 'Big 3 Critical Illness', 
                   'Essential Life': 'Life', 'Cancer-Stroke Insurance': 'Big 3 Critical Illness', 'Modular Maid - 6ME Homebased-Modular Maid - OutPatient': 'Maid', 
                   'Modular Maid - OutPatient': 'Maid', 'Modular Maid - 6ME Clinics': 'Maid', 'Modular Maid - POLO': 'Maid', 'Modular Maid - 6ME Homebased': 'Maid', 
                   'Modular Maid - 6ME Clinics-Modular Maid - OutPatient': 'Maid', 'Modular Maid - 6ME Homebased-Modular Maid - POLO': 'Maid', 
                   'Modular Maid - 6ME Homebased-Modular Maid - OutPatient-Modular Maid - POLO': 'Maid', 'Modular Maid - 6ME Clinics-Modular Maid - POLO': 'Maid', 
                   'Modular Maid - 6ME Clinics-Modular Maid - OutPatient-Modular Maid - POLO': 'Maid', 'Modular Maid - 6ME Homebased-Modular Maid - OutPatient': 'Maid', 
                   'Modular Maid - OutPatient-Modular Maid - POLO': 'Maid'}
df_cleaned['product_type'].replace(mapping_product, inplace=True)

# consolidating genders
mapping_gender = {'Male': 'M', 'm': 'M', 'male': 'M', 'Female': 'F', 'f': 'F', 'Femaile': 'F', 'u': 'U'}
df_cleaned['applicant_gender'].replace(mapping_gender, inplace=True)
df_cleaned['applicant_gender'] = df_cleaned['applicant_gender'].fillna('U')

# consolidating nationalities, incomplete
mapping_nationality = {'Singapore': 'SGP', 'Malaysia': 'MYS'}
df_cleaned['applicant_nationality'].replace(mapping_nationality, inplace=True)

# removed extreme promo code value (100,000)
df_cleaned = df_cleaned[df_cleaned['quotation_promo_code_value'] != 100000]

# consolidating distribution_channel
mapping_distribution = {'AFFINITY': "AFFILIATE", 'SOCIAL': "REFERAFRIEND", 'DISPLAY': 'OTHER', 'WORKSITE':'OTHER', 'EXTERNAL DATABASE': 'OTHER', 'EXTERNALDATABASE': 'OTHER',  'OTO': 'OTHER'}
df_cleaned['distribution_channel'].replace(mapping_distribution, inplace=True)

# fill missing values with 0
df_cleaned['quote_created_before_count'] = df_cleaned['quote_created_before_count'].fillna(0)
df_cleaned['policy_issued_nfc_count'] = df_cleaned['policy_issued_nfc_count'].fillna(0)
df_cleaned['scv_policy_issued_nfc_count'] = df_cleaned['scv_policy_issued_nfc_count'].fillna(0)
df_cleaned['scv_policy_cancelled_count'] = df_cleaned['scv_policy_cancelled_count'].fillna(0)
df_cleaned['scv_policy_inforce_count'] = df_cleaned['scv_policy_inforce_count'].fillna(0)
df_cleaned['scv_policy_gi_count'] = df_cleaned['scv_policy_gi_count'].fillna(0)
df_cleaned['scv_policy_li_count'] = df_cleaned['scv_policy_li_count'].fillna(0)
df_cleaned['quotation_promo_code_value'] = df_cleaned['quotation_promo_code_value'].fillna(0)
df_cleaned['since_first_quote'] = df_cleaned['since_first_quote'].fillna(-1)
df_cleaned['since_latest_quote'] = df_cleaned['since_latest_quote'].fillna(-1)
df_cleaned['since_first_policy'] = df_cleaned['since_first_policy'].fillna(-1)
df_cleaned['since_latest_policy'] = df_cleaned['since_latest_policy'].fillna(-1)
df_cleaned['since_first_interaction'] = df_cleaned['since_first_interaction'].fillna(-1000)
df_cleaned['since_last_interaction'] = df_cleaned['since_last_interaction'].fillna(-1000)

# date_count features
df_cleaned['avg_days_between_quote'] = (df_cleaned['since_first_quote']/df_cleaned['quote_created_before_count']).fillna(-1)
df_cleaned['avg_days_between_policy'] = (df_cleaned['since_first_policy']/df_cleaned['scv_policy_issued_nfc_count']).fillna(-1)

# capped age
df_cleaned['age_at_quote'] = np.where(df_cleaned['age_at_quote']<18,18,(np.where(df_cleaned['age_at_quote']>65,65,df_cleaned['age_at_quote'])))
df_cleaned['age_at_quote'] = df_cleaned['age_at_quote'].fillna(0)

# drop non-useful columns ### CAN INVESTIGATE FURTHER
df_cleaned = df_cleaned.drop(columns=['distribution_channel', 'event_id', 'nric', 'applicant_marital_status', 'applicant_occupation', 'applicant_nationality', 'issued_before_interaction_flag', 'stp_flag', 'nric_quote', 'matched_quote', 'nric_policy', 'matched_policy'])
# df_cleaned = df_cleaned.drop(columns=['quote_valid_flag',])


In [ ]:
CATEGORICAL_COLUMNS = ['product_type', 'device_type', 'applicant_gender', 'nric_type']
for i in CATEGORICAL_COLUMNS:
    df_cleaned = clip_by_prop(df_cleaned, i, 0.05)
    
df_cleaned = pd.get_dummies(df_cleaned, columns = CATEGORICAL_COLUMNS)

# Save Dataset 

In [ ]:
ASSISTED_PATH = "./Data/Assisted_Data"
df_cleaned.to_csv("{}/df_combined.csv".format(ASSISTED_PATH))